# Multiple dispatch

**Multiple dispatch** is a key feature of Julia, that we will explore in this notebook.

It helps make software fast. It also makes software extensible, programmable, and downright fun to play with. 

It may just herald a breakthrough for parallel computation.

1. Roman numerals
2. Functions
3. Parallel computing

## 1. Roman numerals (for fun)

Let's define a **new type** to represent a Roman numeral. For coding simplicity, we'll just deal with numbers between 0 and 9. 

**Exercise**: Extend this to larger numbers. (Recall that Roman numbers are a base-10 system!)

In [ ]:
type Roman
    n::Int
end

Base.show(io::IO, r::Roman) = print(io, 'ⅰ' + (r.n - 1) % 10 )  # nice display; 'ⅰ' is a Unicode Roman numeral

We can create an object of this type as follows:

In [ ]:
Roman(4)

In [ ]:
Roman(14)

We would like to display it nicely, in Roman numerals:

In [ ]:
v = [7 1 2 5 8 9]
Roman.(v)   # equivalent to map(Roman, v)  or  [Roman(x) for x in v]

In [ ]:
Roman(4) + Roman(5)

In [ ]:
import Base.+
+(a::Roman, b::Roman) = Roman(a.n + b.n)

In [ ]:
Roman(4) + Roman(5)

This **adds a new method** to the function `+`:

In [ ]:
methods(+)

In [ ]:
import Base.*

*(i::Roman, j::Roman) = Roman(i.n * j.n)                     # Multiply like a Roman

In [ ]:
Roman(3) * Roman(2)

But 

In [ ]:
Roman(3) * 2

The simplest thing to do is to explicitly define multiplication of a `Roman` by a number. We can do it as we see fit:

In [ ]:
*(i::Number, j::Roman) = eye(j.n) * i       # Multiply like a matrix constructor

*(i::Roman, j::Number) = "😄"^ ((i.n)*j)    # Multiply like a kid

In [ ]:
3 * Roman(3)

In [ ]:
Roman(80) * 10

## Functions

In [ ]:
*(α::Number,   g::Function) = x -> α * g(x)   # Scalar times function

*(f::Function, λ::Number)   = x -> f(λ * x)   # Scale the argument

*(f::Function, g::Function) = x -> f(g(x))    # Function composition  -- abuse of notation!  use \circ in Julia 0.6

In [ ]:
+(f::Function, g::Function) = x -> f(x) + g(x)

For example, the exponential function is defined as

$$\exp(x) = \sum_{n=0}^\infty \frac{1}{n!} x^n.$$

We can think of this just in terms of functions:

$$\exp = \sum_{n=0}^\infty \frac{1}{n!} \mathrm{pow}_n,$$

where $\mathrm{pow}_n(x) = x^n$.

In [ ]:
pow(n) = x -> x^n

myexp = sum(1/factorial(big(n)) * pow(n) for n in 0:30)   # not efficient!

In [ ]:
[myexp(10) exp(10)]

In [ ]:
f = x -> x^2
f(10)

In [ ]:
g = 3f
g(10)

In [ ]:
(f^2)(10)  # since we defined multiplication of functions as composition

In [ ]:
using PyPlot;

In [ ]:
x=pi*(0:.01:4)

plot(x,(12*sin)(x),"g")
plot(x,(sin*12)(x),"r")
plot(x,(5*sin*exp)(x),"b")

<img src="https://lh4.googleusercontent.com/--z5eKJbB7sg/UffjL1iAd4I/AAAAAAAABOc/S_wDVyDOBfQ/gauss.jpg">

###  "Sin^2 phi is odious to me, even though Laplace made use of it; should  it be feared that sin^2 phi might become ambiguous, which would perhaps  never occur, or at most very rarely when speaking of sin(phi^2), well  then, let us write (sin phi)^2, but not sin^2 phi, which by analogy  should signify sin(sin phi)." -- Gauss

In [ ]:
x=(0:.01:2)*pi;
plot(x,(sin^2)(x),"b")     # Squaring just works, y=sin(sin(x)), Gauss would be pleased!
plot(x,sin(x).^2,"r")         

In [ ]:
f(a, b::Any) = "fallback"
f(a::Number, b::Number) = "a and b are both numbers"
f(a::Number, b) = "a is a number"
f(a, b::Number) = "b is a number"
f(a::Integer, b::Integer) = "a and b are both integers"